In [1]:
import cobra,rba,lxml
from lxml import etree

In [2]:
rtGSM = cobra.io.load_json_model('./gsmodel/iRhtoC_r2_2.json')

Academic license - for non-commercial use only


In [3]:
modelET = etree.Element('RBAMetabolism')

elems = ['listOfCompartments', 'listOfSpecies', 'listOfReactions']
for e in elems:
    modelET.append(etree.Element(e))

In [4]:
comps = sorted(rtGSM.compartments.keys())
compET = modelET.find('listOfCompartments')
for comp in comps:
    compET.append(etree.Element('compartment', attrib={'id':'PART-' + comp}))

In [5]:
mets = sorted([met.id for met in rtGSM.metabolites])
metET = modelET.find('listOfSpecies')
for metid in mets:
    met = rtGSM.metabolites.get_by_id(metid)
    comp = met.compartment
    bcond = 'true' if comp == 'e' else 'false'
    att_dict = {'id':'MET-'+met.id, 'boundaryCondition':bcond}
    metET.append(etree.Element('species', attrib=att_dict))

In [6]:
rxns = sorted([rxn.id for rxn in rtGSM.reactions])
rxnET = modelET.find('listOfReactions')

In [7]:
for rxnid in rxns:
    rxn = rtGSM.reactions.get_by_id(rxnid)
    rev = 'true' if rxn.reversibility else 'false'
    att_dict = {'id':'RXN-'+rxn.id, 'reversible':rev}
    rxnE = etree.SubElement(rxnET, 'reaction', attrib=att_dict)
    
    reactants = sorted([met.id for met in rxn.reactants])
    rxnE_reactants = etree.SubElement(rxnE, 'listOfReactants')
    for metid in reactants:
        met = rtGSM.metabolites.get_by_id(metid)
        coeff = rxn.metabolites[met]
        att_dict = {'species':'M_'+met.id, 'stoichiometry':str(coeff)}
        rxnE_reactants.append(etree.Element('speciesReference', attrib=att_dict))
        
    products = sorted([met.id for met in rxn.products])
    rxnE_products = etree.SubElement(rxnE, 'listOfProducts')
    for metid in products:
        met = rtGSM.metabolites.get_by_id(metid)
        coeff = rxn.metabolites[met]
        att_dict = {'species':'M_'+met.id, 'stoichiometry':str(coeff)}
        rxnE_products.append(etree.Element('speciesReference', attrib=att_dict))

In [8]:
with open('./metabolism.xml', 'wb') as f:
    f.write(etree.tostring(modelET, pretty_print=True))